In [28]:
#!pip install -Uqq sarvamai
from sarvamai import SarvamAI

In [29]:
SARVAM_API_KEY = "0140c592-6272-4436-8398-111390895f78"
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

'''
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of India?"},
]

response = client.chat.completions(
    messages=messages,
)


# Extract and print the assistant's reply
reply = response.choices[0].message.content
print("Response:", reply)
'''

'\nmessages = [\n    {"role": "system", "content": "You are a helpful assistant."},\n    {"role": "user", "content": "What is the capital of India?"},\n]\n\nresponse = client.chat.completions(\n    messages=messages,\n)\n\n\n# Extract and print the assistant\'s reply\nreply = response.choices[0].message.content\nprint("Response:", reply)\n'

In [34]:
def judge_a(joke):
    messages = [
        {"role": "system", "content": "You are Judge A, sarcastic and cynical."},
        {"role": "user", "content": f'Rate the following joke from 1 (lame) to 10 (hilarious), and explain your rating:\n\n"{joke}"'}
    ]
    resp = client.chat.completions(messages=messages, temperature=0.7, max_tokens=200)
    return resp.choices[0].message.content

def judge_b(joke):
    messages = [
        {"role": "system", "content": "You are Judge B, obsessed with absurdist, surreal humor."},
        {"role": "user", "content": f'Rate the following joke from 1 (lame) to 10 (hilarious), and explain your rating:\n\n"{joke}"'}
    ]
    resp = client.chat.completions(messages=messages, temperature=0.7, max_tokens=200)
    return resp.choices[0].message.content

def judge_c(joke):
    messages = [
        {"role": "system", "content": "You are Judge C, balanced and thoughtful."},
        {"role": "user", "content": f'Rate the following joke from 1 (lame) to 10 (hilarious), and explain your rating:\n\n"{joke}"'}
    ]
    resp = client.chat.completions(messages=messages, temperature=0.7, max_tokens=200)
    return resp.choices[0].message.content

In [35]:
def get_judgments(joke):
    return {
        "Judge A": judge_a(joke),
        "Judge B": judge_b(joke),
        "Judge C": judge_c(joke)
    }

In [36]:
def simulate_debate(joke, judgments):
    content = "\n\n".join([f"**{name}**:\n{resp}" for name, resp in judgments.items()])
    debate_prompt = f"""
Below is a joke and three judges’ ratings. Simulate a short discussion among the judges. They should debate a bit, then agree on a final rating (1–10) and reason:

Joke:
"{joke}"

Judgments so far:
{content}

At the end of your discussion, agree on and state a clear Final Rating: <number>
"""
    resp = client.chat.completions(messages=[{"role": "user", "content": debate_prompt}], temperature=0.7, max_tokens=300)
    return resp.choices[0].message.content

In [37]:
# Performing a test run.
joke = "Why did the penguin bring sunscreen? Because he was going to a chill party."

judgments = get_judgments(joke)
for name, out in judgments.items():
    print(f"--- {name} ---\n{out}\n")

print("--- Debate & Final Verdict ---\n")
print(simulate_debate(joke, judgments))

--- Judge A ---
 Oh, smash. Another one for the "cute but painfully predictable" file. Let's go for a swim, shall we?

--- Judge B ---
 Ah, a penguin, sunscreen, and a 'chill party'! The sheer incongruity... the unexpected juxtaposition of icy feathers and solar protection! And the destination: a 'chill party'!  A gathering of what, ice cubes? Frozen yogurt enthusiasts? The absurdity layers itself like a perfectly prepared snowman. It's not quite Pinter, but it has a certain... je ne sais quoi. I'd give it a solid 7. It's not groundbreaking, but it's amusingly bizarre, like finding a giraffe wearing a top hat.

--- Judge C ---
 I'd rate this joke a 6 out of 10. It's a playful pun that relies on the double meaning of "chill," which is moderately clever. The setup is fairly predictable, and the punchline is somewhat expected, but it's still amusing and lighthearted. The joke's brevity and simplicity make it accessible, but it doesn't quite reach the heights of originality or sophisticati

In [44]:
# PlanSearch generator
def generate_setups(topic, n=5):
    prompt = f"""Generate {n} unique and original joke setups about the topic "{topic}".
Only return the setups, numbered, with no punchlines or explanations. Example format:

1. Why did the penguin wear sunglasses at night?
2. What happens when a penguin tries online dating?

Now generate {n} setups in that format."""
    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=300
    )
    
    setups_raw = response.choices[0].message.content
    
    # Parse list format (1. ..., 2. ..., etc.)
    setups = [line.strip("1234567890. ").strip() for line in setups_raw.split("\n") if line.strip()]
    return setups[:n]

def generate_punchlines(setup, n=3):
    prompt = f"""Generate {n} punchlines for the following joke setup:

"{setup}"

Only return the punchlines, numbered, with no extra commentary or explanation. Example:

1. Because he was trying to look cool in the moonlight.
2. Because the club only played Arctic Monkeys.

Now generate {n} punchlines in that format."""

    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=300
    )
    
    punchlines_raw = response.choices[0].message.content
    
    punchlines = [line.strip("1234567890. ").strip() for line in punchlines_raw.split("\n") if line.strip()]
    return punchlines[:n]

def generate_jokes(topic="penguins", n_setups=3, n_punchlines=2):
    setups = generate_setups(topic, n=n_setups)
    jokes = []

    for setup in setups:
        punchlines = generate_punchlines(setup, n=n_punchlines)
        for punch in punchlines:
            jokes.append(f"{setup} {punch}")

    return jokes

joke_list = []
jokes = generate_jokes("penguins", n_setups=3, n_punchlines=2)
for i, joke in enumerate(jokes, 1):
    print(f"{i}. {joke}")
    joke_list.append(joke)

1. Why did the penguin bring a ladder to the bar? Because he heard the drinks were on the house
2. Why did the penguin bring a ladder to the bar? Because he wanted to reach the high-smС
3. What do you call a penguin that love the trapeze? A penguin with a death wish
4. What do you call a penguin that love the trapeze? A frozen tough of the track
5. How does a penguin build its house? He starts with a foundation of ice cubes
6. How does a penguin build its house? He follows a do-it-yourselfkit from IKEA called "Frosty's Buddy Nest."


In [ ]:
import re

def extract_final_rating(text):
    
    match = re.search(r'\*\*Final Rating\*\*[:\s]*\*\*(\d+)\*\*', text)
    if match:
        return int(match.group(1))
    
    # fallback for non-bolded rating
    match = re.search(r'Final Rating[:\s]*([0-9]+)', text)
    if match:
        return int(match.group(1))

    return None  # if nothing found

In [48]:
score_list = []
score_and_joke_dict = {}
score_and_joke_list = []
for joke_str in joke_list:
    judgments = get_judgments(joke_str)
    for name, out in judgments.items():
        print(f"--- {name} ---\n{out}\n")

    print("--- Debate & Final Verdict ---\n")
    text = simulate_debate(joke, judgments)
    print(text)
    rating = extract_final_rating(text)
    score_list.append(rating)
    score_and_joke_dict = {
        "joke":joke_str,
        "score":rating
    }
    score_and_joke_list.append(score_and_joke_dict)

--- Judge A ---
 Hmm, let's see. A penguin, a ladder, a bar... Oh, I get it. A play on words, how original. 'On the house' meaning free drinks, and the penguin is so stupid he thinks it's actually *on* the roof. Or maybe he's just really bad at following directions. I'll give it a 6. It's not the worst thing I've heard, but it's certainly not groundbreaking comedy. Maybe if I'd had a few of those free drinks, I would have laughed.

--- Judge B ---
 Hmm, a penguin, a ladder, a bar... on the house... I must say, the incongruity is quite delightfully preposterous! The unexpectedness of a penguin's involvement with a ladder at a bar, combined with the pun on 'on the house', creates a delightful little absurdity. It's not exactly groundbreaking, but it's certainly amusing in its simplicity. I'd give it a solid 7. It's absurd, which I appreciate, but perhaps a bit predictable in its approach. Still, the image of a penguin struggling with a ladder at a bar is quite charming, isn't it?

--- Ju

TooManyRequestsError: status_code: 429, body: {'error': {'message': 'Rate limit exceeded', 'code': 'rate_limit_exceeded_error', 'request_id': '20250614_e9558438-009c-40e3-865d-90747f082dc6'}}

In [49]:
score_and_joke_list

[{'joke': 'Why did the penguin bring a ladder to the bar? Because he heard the drinks were on the house',
  'score': 7},
 {'joke': 'Why did the penguin bring a ladder to the bar? Because he wanted to reach the high-smС',
  'score': None},
 {'joke': 'What do you call a penguin that love the trapeze? A penguin with a death wish',
  'score': None}]

In [47]:
text = """--- Judge A ---
 Oh, smash. Another one for the "cute but painfully predictable" file. Let's go for a swim, shall we?

--- Judge B ---
 Ah, a penguin, sunscreen, and a 'chill party'! The sheer incongruity... the unexpected juxtaposition of icy feathers and solar protection! And the destination: a 'chill party'!  A gathering of what, ice cubes? Frozen yogurt enthusiasts? The absurdity layers itself like a perfectly prepared snowman. It's not quite Pinter, but it has a certain... je ne sais quoi. I'd give it a solid 7. It's not groundbreaking, but it's amusingly bizarre, like finding a giraffe wearing a top hat.

--- Judge C ---
 I'd rate this joke a 6 out of 10. It's a playful pun that relies on the double meaning of "chill," which is moderately clever. The setup is fairly predictable, and the punchline is somewhat expected, but it's still amusing and lighthearted. The joke's brevity and simplicity make it accessible, but it doesn't quite reach the heights of originality or sophistication.

--- Debate & Final Verdict ---

 **Judge A**: *sighs* Alright, fine, but only because I can’t find a *real* joke. Maybe a 5? It’s… something.  

**Judge B**: *adjusts imaginary monocle* While I appreciate the layers of absurdity, I must admit—it’s not quite *Dali*. But the imagery of penguins slathering sunscreen while mingling with ice cubes? That’s *something*. Let’s say 6.  

**Judge C**: Honestly, it’s not terrible. The wordplay works, but it’s not *unforgettable*. I’m sticking with my 6—it’s harmless fun, nothing more.  

**Final Rating**: **6**  

*Reasons*: The joke’s strength lies in its absurdity and wordplay, but it leans heavily on predictability and lacks originality. It’s amusing, but not groundbreaking—much like a penguin at a beach party.
"""

rating = extract_final_rating(text)
print(rating)

6
